In [22]:
from __future__ import print_function
import tensorflow as tf
from classification.models import model
from classification import metadata
import numpy as np
from datetime import datetime
import pytz
from classification import run_inference
import json
import pandas
import matplotlib.pyplot as plt
import subprocess
import pandas as pd
%matplotlib inline

In [9]:
training_df = pandas.read_csv('classification/data/training_classes_vessel_id.csv')

//anaconda/envs/ml2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (12,18,19,20,22,23,24,26,27,29,31,33,34,35,36,37,38,39,40,41,42,43) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [43]:
df_subset = training_df[['mmsi', 'label', 'length', 'tonnage', 'engine_power', 'crew_size', 'split']]
df_subset.to_csv('training_classes_vessel_id_subset.csv', index=False)
subprocess.check_call(['gsutil', 'cp', 'training_classes_vessel_id_subset.csv', 
                      'gs://machine-learning-dev-ttl-120d/training_classes_vessel_id_v20181025.csv'])

0

In [45]:
getattr

<function getattr>

In [35]:
inference_table = 'world-fishing-827.machine_learning_dev_ttl_120d.smoke_test_vessel_inference_v20181024_20170701'
label_table = "world-fishing-827.machine_learning_dev_ttl_120d.training_classes_vessel_id_v20181025"
query = """
SELECT a.* FROM 
`{}` a
JOIN
`{}` b
ON a.vessel_id = b.mmsi 
LIMIT 100
""".format(inference_table, label_table)
inference_df = pd.read_gbq(query, project_id='world-fishing-827', dialect='standard')

In [41]:
label_df = pd.read_gbq("select * from `{}`".format(label_table), project_id='world-fishing-827', dialect='standard')

In [44]:
inference_df.head()

,vessel_id,start_time,end_time,max_label,length,tonnage,engine_power,crew_size,label_scores
0,ece91e16f-fd8d-13cf-892e-55b112ee6a99,2017-07-01,2017-12-28,tug,33.365195,277.756079,740.284230,16.420634,"[{u'score': 7.03e-08, u'label': u'bunker'}, {u..."
1,fff2a55a1-18de-5536-0459-85903e96ccd6,2017-07-01,2017-12-28,tug,17.293510,28.936498,217.801276,10.701784,"[{u'score': 2.4e-09, u'label': u'bunker'}, {u'..."
2,6c30b7196-6263-cea5-90ae-fa3646a94d12,2017-07-01,2017-12-28,tug,30.903055,161.798137,562.024985,10.799887,"[{u'score': 3.6e-09, u'label': u'bunker'}, {u'..."
3,9cdf67528-88a4-4431-dc18-a4404db60283,2017-07-01,2017-12-28,tug,37.762387,181.600188,1358.129752,14.452969,"[{u'score': 7.896e-07, u'label': u'bunker'}, {..."
4,048028a49-9520-bb4e-86f9-802248bbd570,2017-07-01,2017-12-28,tug,31.498599,410.723605,1187.674468,7.707851,"[{u'score': 1.74e-08, u'label': u'bunker'}, {u..."


In [37]:
'''
python -m classification.metrics.compute_vessel_metrics \
     --inference-table  machine_learning_dev_ttl_120d.smoke_test_vessel_inference_v20181024_ \
     --label-table world-fishing-827.machine_learning_dev_ttl_120d.training_classes_vessel_id_v20181025 \
     --dest-path test_new_vessel_inference.html
'''

[{u'label': u'bunker', u'score': 7.03e-08},
 {u'label': u'cargo', u'score': 0.0394062512},
 {u'label': u'container_reefer', u'score': 1.692e-07},
 {u'label': u'dive_vessel', u'score': 0.0},
 {u'label': u'dredge_fishing', u'score': 0.000176081},
 {u'label': u'dredge_non_fishing', u'score': 0.0348936953},
 {u'label': u'drifting_longlines', u'score': 6.2658e-05},
 {u'label': u'driftnets', u'score': 3e-09},
 {u'label': u'fish_factory', u'score': 3.36222e-05},
 {u'label': u'gear', u'score': 8.86e-08},
 {u'label': u'helicopter', u'score': 0.0},
 {u'label': u'other_fishing', u'score': 6.84e-07},
 {u'label': u'other_not_fishing', u'score': 0.0280245729},
 {u'label': u'other_purse_seines', u'score': 0.0003788223},
 {u'label': u'other_seines', u'score': 3.9729e-06},
 {u'label': u'passenger', u'score': 0.0075969142},
 {u'label': u'patrol_vessel', u'score': 0.0003363299},
 {u'label': u'pole_and_line', u'score': 1.6973e-06},
 {u'label': u'pots_and_traps', u'score': 1.8917e-05},
 {u'label': u'resear

In [2]:
class MyModel(model.ModelBase): 
    def build_inference_net(self):
        pass
    def build_training_net(self):
        pass
    @property
    def max_window_duration_seconds(self):
        # A fixed-length rather than fixed-duration window.
        return 0

    @property
    def window_max_points(self):
        return 1024
    
    def make_input_fn(self, base_feature_path, split, num_parallel_reads):
        def training_input_fn():
            return (fishing_feature_generation.input_fn(
                        self.vessel_metadata,
                        self.build_training_file_list(base_feature_path, split),
                        self.num_feature_dimensions + 1,
                        self.max_window_duration_seconds,
                        self.window_max_points,
                        self.min_viable_timeslice_length,
                        select_ranges=self.use_ranges_for_training,
                        num_parallel_reads=num_parallel_reads)
                .prefetch(self.batch_size)
                .batch(self.batch_size)
                )
        return training_input_fn
    
    def make_training_input_fn(self, base_feature_path, num_parallel_reads):
        return self.make_input_fn(base_feature_path, utility.TRAINING_SPLIT, num_parallel_reads)
    
root_feature_path = "gs://machine-learning-dev-ttl-120d/features/ppb_features_through2017/features"
fishing_range_file = "classification/data/combined_fishing_ranges_vessel_id.csv"
metadata_file = "classification/data/training_classes_vessel_id.csv"

fishing_ranges = metadata.read_fishing_ranges(fishing_range_file)
all_available_mmsis = metadata.find_available_mmsis(root_feature_path)

vessel_metadata = MyModel.read_metadata(
        all_available_mmsis, metadata_file,
        fishing_ranges, 1)

# mdl = MyModel(14, vessel_metadata)
# files = mdl.build_training_file_list(root_feature_path , utility.TRAINING_SPLIT)

In [3]:
sorted(vessel_metadata.metadata_by_mmsi)[:10]

['000020b39-9a62-1a09-60fe-d65a9e39c2cf',
 '0000c5d13-3506-2b61-4e28-d6899b26042a',
 '0000e640f-f180-f001-2eec-7e35ff334468',
 '0000f9035-5cdc-333b-2a65-5cbb36b11e7e',
 '00012e8f4-4969-4ade-4c87-441dd4df3cb9',
 '0001caf0c-ccad-da52-51da-89a663aa557d',
 '0001d40eb-b6cd-6b77-df8d-abfb50da79ef',
 '00023cd36-6923-af32-1bef-62ccd5ae74f1',
 '000247f64-4b77-df37-ad93-f61ef44e72b3',
 '000257665-5945-b990-1011-bc453452ae78']

In [4]:
import imp
import pytz
import classification.models.objectives
imp.reload(classification.models.objectives)
from classification.feature_generation import vessel_feature_generation
imp.reload(vessel_feature_generation)
from classification.feature_generation import fishing_feature_generation
imp.reload(fishing_feature_generation)
import classification.models.vessel_characterization
imp.reload(classification.models.vessel_characterization)
import classification.models.vessel_characterization
imp.reload(classification.models.vessel_characterization)
from classification.models.vessel_characterization import Model
import classification.models.fishing_detection
import classification.feature_generation.feature_utilities
imp.reload(classification.feature_generation.feature_utilities)
from classification.feature_generation.feature_utilities import EPOCH_DT
# imp.reload(classification.models.prod.fishing_detection)
# from classification.models.prod.fishing_detection import Model

mdl = Model(14, vessel_metadata, 'minimal')
    
time_ranges = [(1328083200, 1343635200), (1343808000, 1359360000), (1359705600, 1375257600), (1375344000, 1390896000), (1391241600, 1406793600), (1406880000, 1422432000), (1422777600, 1438329600), (1438416000, 1453968000), (1454313600, 1469865600), (1470038400, 1485590400), (1485936000, 1501488000), (1501574400, 1517126400)]
# time_ranges = [((datetime(2015, 6, 1, tzinfo=pytz.utc) - EPOCH_DT).total_seconds(), (datetime(2015, 7, 1, tzinfo=pytz.utc) - EPOCH_DT).total_seconds())]
template = "gs://machine-learning-dev-ttl-120d/features/ppb_features_through2017/features/{}.tfrecord"
vessel_ids = vessel_metadata.metadata_by_mmsi.keys()[:10]
paths = [template.format(x) for x in vessel_ids]

# paths = """
# gs://machine-learning-dev-ttl-120d/features/ppb_features_through2017/features/000007c49-9673-3128-434d-6937d3400dd3.tfrecord
# gs://machine-learning-dev-ttl-120d/features/ppb_features_through2017/features/000008a58-83b9-f7c6-e3af-148f962497f4.tfrecord
# gs://machine-learning-dev-ttl-120d/features/ppb_features_through2017/features/000020b39-9a62-1a09-60fe-d65a9e39c2cf.tfrecord
# gs://machine-learning-dev-ttl-120d/features/ppb_features_through2017/features/000027eb4-4bec-76cc-062b-6ce5cdcac685.tfrecord
# gs://machine-learning-dev-ttl-120d/features/ppb_features_through2017/features/000073b65-58c6-abce-7fb1-a842a589aa96.tfrecord
# gs://machine-learning-dev-ttl-120d/features/ppb_features_through2017/features/0000747d7-71d7-a018-9e87-d0fbab2adac6.tfrecord
# gs://machine-learning-dev-ttl-120d/features/ppb_features_through2017/features/00009b2f0-0822-cdfc-ba1e-a4b05787f7b6.tfrecord
# gs://machine-learning-dev-ttl-120d/features/ppb_features_through2017/features/00009f31c-ca68-b94d-11cc-f47ebd7390b8.tfrecord
# gs://machine-learning-dev-ttl-120d/features/ppb_features_through2017/features/0000a8ba8-8c4c-255c-59e6-de6c90f79862.tfrecord
# gs://machine-learning-dev-ttl-120d/features/ppb_features_through2017/features/0000b163b-b6ee-1b90-63d2-a2a1c6864e5a.tfrecord
# """.strip().split()

range_info = (datetime(2017,1,1), datetime(2017, 7, 1))
input_fn_p = mdl.make_prediction_input_fn(paths, time_ranges, 32)
iter_p = input_fn_p().make_one_shot_iterator()
el_p = iter_p.get_next()

input_fn_t = mdl.make_training_input_fn(root_feature_path, 32)
iter_t = input_fn_t().make_one_shot_iterator()
el_t = iter_t.get_next()

with tf.Session() as sess:
#     while True:
        x_p = sess.run(el_p)
        x_t = sess.run(el_t)
        print(x_p)
        print(x_t)


{'timestamps': array([[1448952064, 1448997760, 1449019776, ..., 1453312256, 1453313280,
        1453314176]], dtype=int32), 'mmsi': array(['-4484213255844134909'], dtype=object), 'features': array([[[10.730335  ,  6.601677  ,  0.41504836, ...,  0.        ,
          0.        ,  0.        ],
        [ 9.997342  ,  7.2467427 ,  0.7074879 , ...,  0.        ,
          0.        ,  0.        ],
        [ 8.23483   ,  6.0153675 ,  0.34469208, ...,  0.        ,
          0.        ,  0.        ],
        ...,
        [ 6.929517  ,  4.9091506 ,  0.41504836, ...,  0.        ,
          0.        ,  0.        ],
        [ 6.8669333 ,  5.94337   ,  0.38048884, ...,  0.        ,
          0.        ,  0.        ],
        [ 6.8679743 ,  6.0894523 ,  0.14352287, ...,  0.        ,
          0.        ,  0.        ]]], dtype=float32), 'time_ranges': array([[1438416000, 1453968000]], dtype=int32)}
({'timestamps': array([[1430546816, 1430547584, 1430548352, ..., 1430617856, 1430618496,
        143061

In [5]:
x_p['features'].shape
print(x_p['features'].mean(axis=(0, 1)))
print(x_p['features'].std(axis=(0, 1)))

[ 6.6799445e+00  4.3288317e+00  4.1018948e-01  3.8333207e-01
  4.1953898e-03  4.1828036e-02 -2.6415611e-02  9.2504042e-01
  2.8000000e-01  1.3129959e-02  0.0000000e+00  0.0000000e+00
  0.0000000e+00  0.0000000e+00]
[0.77054167 2.6774557  0.5551633  0.49860933 0.4567945  0.7275532
 0.68426394 0.06650381 0.24820158 0.5663661  0.         0.
 0.         0.        ]


In [6]:
print(x_t[0]['features'].mean(axis=(0, 1)))
print(x_t[0]['features'].std(axis=(0, 1)))

[ 6.8813524e+00  4.6720781e+00  7.0673662e-01  6.5744787e-01
 -1.0784438e-03 -9.3182698e-02 -4.9768318e-04 -4.9298774e-02
  7.6009883e-03 -5.9309457e-03  1.7335727e+00  0.0000000e+00
  0.0000000e+00  0.0000000e+00]
[0.91939354 3.3832622  0.76762664 0.7578671  0.36444297 0.7101269
 0.697734   0.6980936  0.71488523 0.49576244 1.8275083  0.
 0.         0.        ]


In [7]:
chkpt_path = 'gs://world-fishing-827-dev-ttl30d/data-production/classification/timothyhochberg/vessel_char_v20181023A/models/vessel_characterization'
estimator = mdl.make_estimator(chkpt_path)

INFO:tensorflow:Using config: {'_save_checkpoints_secs': 300, '_global_id_in_cluster': 0, '_session_config': allow_soft_placement: true
, '_keep_checkpoint_max': 10, '_tf_random_seed': None, '_task_type': 'worker', '_train_distribute': None, '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x14a360450>, '_model_dir': 'gs://world-fishing-827-dev-ttl30d/data-production/classification/timothyhochberg/vessel_char_v20181023A/models/vessel_characterization', '_num_worker_replicas': 1, '_task_id': 0, '_log_step_count_steps': 100, '_master': '', '_save_checkpoints_steps': None, '_keep_checkpoint_every_n_hours': 10000, '_evaluation_master': '', '_service': None, '_device_fn': None, '_save_summary_steps': 20, '_num_ps_replicas': 0}


INFO:tensorflow:Using config: {'_save_checkpoints_secs': 300, '_global_id_in_cluster': 0, '_session_config': allow_soft_placement: true
, '_keep_checkpoint_max': 10, '_tf_random_seed': None, '_task_type': 'worker', '_train_distribute': None, '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x14a360450>, '_model_dir': 'gs://world-fishing-827-dev-ttl30d/data-production/classification/timothyhochberg/vessel_char_v20181023A/models/vessel_characterization', '_num_worker_replicas': 1, '_task_id': 0, '_log_step_count_steps': 100, '_master': '', '_save_checkpoints_steps': None, '_keep_checkpoint_every_n_hours': 10000, '_evaluation_master': '', '_service': None, '_device_fn': None, '_save_summary_steps': 20, '_num_ps_replicas': 0}


In [8]:
for result in results:
    vid = vessel_metadata.mmsi_map_int2str[int(result['mmsi'])]
    cls = metadata.VESSEL_CLASS_DETAILED_NAMES[np.argmax(result['Vessel-class'])]
    print(vid[:5], 
          vessel_metadata.metadata_by_mmsi[vid][0]['label'], cls,
          vessel_metadata.metadata_by_mmsi[vid][0]['length'], np.exp(result['Vessel-length']))

NameError: name 'results' is not defined

In [ ]:
# So running predictions "by hand" works. What about using an Inferer?

inferer = run_inference.Inferer(mdl, chkpt_path, root_feature_path)

all_results = []
for results in inferer.run_inference(vessel_ids, 6, datetime(2015,1,1, tzinfo=pytz.UTC), 
                                     datetime(2017, 12, 31, tzinfo=pytz.UTC)):
    all_results.append(results)

In [ ]:
for result in all_results:
    vid = vessel_metadata.mmsi_map_int2str[int(result['mmsi'])]
    cls = result['Multiclass']['max_label']
    print(vid[:5], 
          vessel_metadata.metadata_by_mmsi[vid][0]['label'], cls,
          vessel_metadata.metadata_by_mmsi[vid][0]['length'], result['length']['value'])

In [ ]:
cmd = r'''
docker-compose run vessel_inference  \
                --feature_path {}   \
                --checkpoint_path  {}    \
                --feature_dimensions 14   \
                --results_table=world-fishing-827:machine_learning_dev_ttl_120d.smoke_test_vessel_inference_v20181024_   \
                --start_date 2017-01-01   \
                --end_date 2017-12-31   \
                --project world-fishing-827   \
                --temp_location gs://machine-learning-dev-ttl-30d/scratch/inference   \
                --job_name smoke-test-vessel-inference   \
                --max_num_workers 100   \
                --setup_file ./setup.py   \
                --requirements_file requirements.txt   \
                --runner DataflowRunner  \
                --max_num_workers 100  \
                --worker_machine_type=custom-1-13312-ext
'''.format(root_feature_path, chkpt_path)
print(cmd)

In [ ]:
test_vessel_ids = vessel_metadata.metadata_by_split['Test'].keys()
np.random.seed(888)
test_vessel_ids = np.random.choice(test_vessel_ids, 2000)

query = """
SELECT * FROM `world-fishing-827.machine_learning_dev_ttl_120d.smoke_test_vessel_inference_v20181024_*` 
WHERE _TABLE_SUFFIX >= "20170101" AND
vessel_id  in ({})
LIMIT 1000
""".format(', '.join(['"{}"'.format(x) for x in test_vessel_ids]))



results_df = pandas.read_gbq(query, project_id='world-fishing-827', dialect='standard')

results_df.head()

In [ ]:
for result in results_df.itertuples():
    vid = result.vessel_id
    cls = result.max_label
    print(vid[:5], 
          vessel_metadata.metadata_by_mmsi[vid][0]['label'], cls,
          vessel_metadata.metadata_by_mmsi[vid][0]['length'], result.length)

In [ ]:
keys = [x['label'] for x in results_df.iloc[0].label_scores]
x = []
y = []
for result in results_df.itertuples():
    y.append(keys.index(result.max_label))
    vid = result.vessel_id
    lbl = vessel_metadata.metadata_by_mmsi[vid][0]['label']
    x.append(keys.index(lbl))
    
plt.figure(figsize=(12, 12))
plt.plot(x, y, '.', markersize=20, alpha = 0.1)
# keys

In [ ]:
x = []
y = []
for result in results_df.itertuples():
    vid = result.vessel_id
    length = vessel_metadata.metadata_by_mmsi[vid][0]['length']
    if length:
        y.append((result.length))
        vid = result.vessel_id
        length = vessel_metadata.metadata_by_mmsi[vid][0]['length']
        x.append(length)
    
plt.figure(figsize=(12, 12))
plt.plot(x, y, '.', markersize=5)
len(x)

In [ ]:
input_fn = mdl.make_test_input_fn(root_feature_path, 32, prefetch=1)
dataset = input_fn()
iter = dataset.make_one_shot_iterator()
el = iter.get_next()
with tf.Session() as sess:
    el = sess.run(el)

In [ ]:
from classification.models.fishing_detection import Model
real_mdl = Model(14, vessel_metadata, 'minimal')

In [ ]:
dataset = real_mdl.make_test_input_fn(root_feature_path, 4)()
iter = dataset.make_one_shot_iterator()
el = iter.get_next()
with tf.Session() as sess:
    el = sess.run(el)
el[0][0].shape, el[1].shape

In [ ]:
import pandas as pd
train_df = pd.read_csv('classification/data/training_classes_vessel_id.csv')
train_df.head()

In [ ]:
mmsis = open('mmsis.txt').read().strip().split()

In [ ]:
in_both = set(mmsis) & set(train_df.mmsi)
len(in_both)

In [ ]:
sorted(mmsis)[:10]

In [ ]:
sorted(set(train_df.mmsi))[:10]

In [ ]:
mapper = pd.read_csv('train/ssvid_to_vessel_id.csv')

In [ ]:
sorted(set(mapper.vessel_id))[:10]

In [ ]:
def fakerator():
    while True:
        for i in range(3):
            yield i
        raise StopIteration
        
f = fakerator()

print('a')
for x in f:
    print x
    
print('b')
for x in f:
    print x

In [ ]:
2 + 2